### Part a

(a) Consider first a retailer that is not using blockchain. Their consumer demand is stochastic and freshness-dependent: $D = \lambda + \beta X+ \epsilon$, where $\lambda$ is a baseline level of demand, $\beta$ is sensitivity of demand to freshness, and $X$ and $\epsilon$ are independent normally distributed random variables, such that $X \sim \mathcal{N}(0, \sigma_X^2)$ and $\epsilon \sim \mathcal{N}(0, \sigma^2)$. The variable $\epsilon$ describes an idiosyncratic demand shock and the variable $X$ describes uncertain freshness of supply. For simplicity the variable X is mean-centred at zero.

Assume that the baseline level of demand is $\lambda = 357.46$ kg, sensitivity is $\beta = 82.38$ kg/day, and $\sigma=44.72$ and $\sigma_X=0.6509$. 

Assume also that the retailer buys its stock with wholesale price 1 eur/kg and sells it with retail price 3 eur/kg. There is no salvage value with the stock that is unsold. The retailer is risk-neutral and maximises profit. Use the model to answer the following questions: what is the optimal level of fresh food to stock, and what is the associated expected profit with that level?


In [4]:
import numpy as np
np.random.seed(1)  

In [20]:
lambda_ = 357.46  # Baseline demand
beta = 82.38  # Sensitivity of demand to freshness
sigma = 44.72  # Std of demand shock
sigma_X = 0.6509  # Std of freshness
wholesale_buying_price = 1  # EUR/kg
retail_selling_price = 3  # EUR/kg

def demand(X, epsilon):
    return lambda_ + beta * X + epsilon

def profit(stock_level, demands):
    sales = np.minimum(demands, stock_level)
    return np.mean((retail_selling_price - wholesale_buying_price) * sales - 
                   wholesale_buying_price * stock_level)

# Simulation
N = 200000
optimal_stock_level_noblockchain = 0
max_expected_profit_noblockchain = -np.inf

stock_levels = np.arange(1, 1000, 1) 


for stock_level in stock_levels:
    X = np.random.normal(0, sigma_X, N)
    epsilon = np.random.normal(0, sigma, N)
    demands = demand(X, epsilon)
    current_profit = profit(stock_level, demands)
    if current_profit > max_expected_profit_noblockchain:
        max_expected_profit_noblockchain = current_profit
        optimal_stock_level_noblockchain = stock_level

print(f"Best stock level (no blockchain): {optimal_stock_level_noblockchain} kg")
print(f"Expected profit (no blockchain): {max_expected_profit_noblockchain:.2f} euros")

Best stock level (no blockchain): 354 kg
Expected profit (no blockchain): 301.81 euros


### Part b

Now consider a Blockchain retailer who receives freshness information, i.e. knows the realisation  $X=x$ before ordering. Note that even in this case the supply order may not match demand because of the component $\epsilon$ whose realisation is not observed before decision making. Use the model to answer the following questions: What is the improvement in expected profit with a retailer that is using Blockchain? How much less food waste is produced?

Hint: simulate values of $X$ and $\epsilon$ to calculate demand. The retailer can only profit from the stock that is sold, i.e. overage stock is costly but does not contribute to profit. The problem is similar to the hotel room optimisation problem from Assignment 1.

In [22]:
lambda_ = 357.46  # Baseline demand
beta = 82.38  # Sensitivity of demand to freshness
sigma = 44.72  # Std of demand shock
sigma_X = 0.6509  # Std of freshness
wholesale_buying_price = 1  # EUR/kg
retail_selling_price = 3  # EUR/kg

def profit_with_known_X(stock_level, X, epsilon):
    demands = lambda_ + beta * X + epsilon
    sales = np.minimum(demands, stock_level)
    return np.mean((retail_selling_price - wholesale_buying_price) * sales - 
                   wholesale_buying_price * stock_level)

# Simulation
N = 10000 
expected_profits = []

X_sims = np.random.normal(0, sigma_X, N)
epsilon_sims = np.random.normal(0, sigma, N)
stock_levels = np.arange(200, 500, 1) 

for X in X_sims:
    optimal_stock_level_X = 0
    max_expected_profit_X = -np.inf
    for stock_level in stock_levels:
        current_profit = profit_with_known_X(stock_level, X, epsilon_sims)
        if current_profit > max_expected_profit_X:
            max_expected_profit_X = current_profit
            optimal_stock_level_X = stock_level
    expected_profits.append(max_expected_profit_X)

average_expected_profit_blockchain = np.mean(expected_profits)

improvement_in_profit = average_expected_profit_blockchain - max_expected_profit_noblockchain

print(f"Expected profit (no blockchain): {max_expected_profit_noblockchain:.2f} euros")
print(f"Expected profit (with blockchain): {average_expected_profit_blockchain:.2f} euros")
print(f"Improvement in expected profit with Blockchain: {improvement_in_profit:.2f} euros")

Expected profit (no blockchain): 301.81 euros
Expected profit (with blockchain): 321.56 euros
Improvement in expected profit with Blockchain: 19.75 euros


In [27]:
lambda_ = 357.46  # Baseline demand
beta = 82.38  # Sensitivity of demand to freshness
sigma = 44.72  # Std of demand shock
sigma_X = 0.6509  # Std of freshness
wholesale_buying_price = 1  # EUR/kg
retail_selling_price = 3  # EUR/kg

def demand(X, epsilon):
    return lambda_ + beta * X + epsilon

N = 200000 

X_sims = np.random.normal(0, sigma_X, N)
epsilon_sims = np.random.normal(0, sigma, N)

unsold_stock_noblockchain = []
unsold_stock_blockchain = []

stock_levels = np.arange(200, 500, 1)  

# Unsold stock (no blockchain)
for i in range(N):
    d = demand(0, epsilon_sims[i])  # X is 0 because we do not know the freshness
    unsold_stock_noblockchain.append(max(0, optimal_stock_level_noblockchain - d))

# Unsold stock (with blockchain)
for i in range(N):
    X = X_sims[i]
    optimal_stock_level_X = 0
    max_expected_profit_X = -np.inf
    for stock_level in stock_levels:
        d = demand(X, epsilon_sims[i]) 
        profit = (3 - 1) * min(stock_level, d) - 1 * stock_level
        if profit > max_expected_profit_X:
            max_expected_profit_X = profit
            optimal_stock_level_X = stock_level
    d = demand(X, epsilon_sims[i])
    unsold_stock_blockchain.append(max(0, optimal_stock_level_X - d))

average_unsold_noblockchain = np.mean(unsold_stock_noblockchain)
average_unsold_blockchain = np.mean(unsold_stock_blockchain)
food_waste_reduction = average_unsold_noblockchain - average_unsold_blockchain

print(f"Average unsold stock (no blockchain): {average_unsold_noblockchain:.2f} kg")
print(f"Average unsold stock (with blockchain): {average_unsold_blockchain:.2f} kg")
print(f"Reduction in food waste with Blockchain: {food_waste_reduction:.2f} kg")


Average unsold stock (no blockchain): 16.20 kg
Average unsold stock (with blockchain): 0.42 kg
Reduction in food waste with Blockchain: 15.79 kg
